# MLOPS Kubernetes and Kubeflow Hands On Notebook

## Kubernetes Fundamentals

### Overview of Containers and Docker

#### Task: Build a simple Docker image for a Python application and run it locally.
---
**Solution:** 
* Simple FastAPI web application ---> can be found in the `./webapp/` path
* Dockerfile to build the image ---> can be found at `./Dockerfile`
* Commands to run the fastAPI application using Docker.
    ```
    $ docker build -t fastapi_sample_app .
    $ docker run -p 8080:8080 fastapi_sample_app
    ```
* Open `localhost:8080` in the browser to access the application (or use postman to test the API)


### Intro. to Kubernetes : Architecture and Components

#### Task: Explore a running Kubernetes cluster using kubectl (get nodes, pods)
----
* `kubectl get pods`

![get pods](images/pods.png)

* `kubectl get svc`

![get svc](images/svc.png)

* `kubectl get nodes`
* `kubectl get deployments`

etc...

### Setting up kubernetes cluster (Minikube/Kind) 

#### Task: Install Minikube and create a local kubernetes cluster
---

To install Minikube on Windows (WSL2):
```
   > curl -Lo minikube https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64

   > chmod +x ./minikube
   > sudo mv ./minikube /usr/local/bin/
   > minikube config set driver docker

```
Next install Kubectl and use minikube kubectl as default kubectl
```
   > curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"
   > chmod +x ./kubectl
   > sudo mv ./kubectl /usr/local/bin/
   > kubectl config use-context minikube
   > alias kubectl="minikube kubectl --"
```


### Hands-on: Deploy your first Application on Kubernetes

#### Task 1: Create a simple web app and deploy it as K8s deployment.
---
**Solution:**
1. In my case I am using WSL2 inside windows, hence I need to point minikube to the correct docker engine inside minikube.
2. Run below command in terminal (Command can be found in minikube docs)
```
    > eval $(minikube -p minikube docker-env)
```
3. Build the Docker image again (alternatively we can pull the local built image from the local registry, but for simplicity I will just build this image again inside minikube). Also giving a tag to the image is important as it causes issues when creating deployment.
4. Run below command in terminal (Command can be found in minikube docs)
```
    > docker build -t sample_fast_api:v2 .
```
5. Create the deployment
```
    > kubectl create deployment app-dep --image=sample_fast_api:v2
```
6. Now the application is ready and you can check the deployment my using the `kubectl get pods` command to view the pod exactly. or use `kubectl get deploy` to view the deployments.


---
#### Task 2: Expose the application using NodePort or LoadBalancer service. 
---
**Solution:**
1. Now tht we have a running application, we need to expose the application using NodePort or LoadBalancer service. For this I will use the NodePort service.
2. Run below command in terminal (Command can be found in minikube docs)
```
    > kubectl expose deployment app-dep --type=NodePort --port=8080
```
3. Now the service is created, to test this locally using minikube you can expose it using the below commnad:
```
    > minikube service app-dep
```
4. We should see something like this:

    ![minikube service](images/minikube_service.png)

---

## Kubernetes Objects

### Pods, Deployments and Services

#### Task 1: Hands-on: Create a deployment with multiple replicas of a pod.
---

**Solution:**
1. I have create a deployment and a service in a single file ---> `./app-dep-deployment`
2. This will have 2 replicas, and created a nodeport serive on the port 30000 mapping the container port of 8080.
3. create the deplyment using `kubectl create -f app-dep-deployment`
4. Because I am using docker desktop's engine here inside WSL2 we need to run the `minikube service app-dep-service` command to expose the service to test locally.

#### Task 2: Expose the deployment as a service and access it.
---

**Solution:**
1. This is already achieved in the previous step, we will see something like this:

    ![minikube service](images/service_nodeport.png)



### ConfigMaps and Secrets

#### Task 1: Create a Configmap to store application configuration and a secret to store sensitive information.
---
**Solution:**
1. I have created a new YAML file name `app-dep-secret-configmap,yaml`, this contains a secret and a configmap. The deployment will be similar to the previous task, this time I have rebuilt the system with a new docker build as I have made a few changes to the fastAPI application to verify if we are able to access these values as environment variables.

    ![newly added settings](images/secret_configmap.png)
2. Run `kubectl create -f app-dep-secret-configmap.yaml`
3. As we are in WSL2 run `minikube service app-dep-service`

  
---

#### Task 2: Mount ConfigMap and Secret as environment variable in the pod.
---
4. If we hit the `/secret` and `/configMap` endpoints, we should see the values  in the response. These were set as env variables in the deployment.
  ![configmap output](images/configMap.png)
---